In [5]:
# Imports
import tensorflow as tf
import numpy as np

import state
import stateFactory

In [2]:
# Create model of the Neural Network

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(9, 9, 9)), # Input layer
    tf.keras.layers.Dense(units=729, activation='relu'), # Hidden layer
    tf.keras.layers.Dense(units=729, activation='relu'), # Hidden layer
    tf.keras.layers.Dense(units=729, activation='relu'), # Output layer
])

In [3]:
# Compiling model

model.compile(loss='binary_crossentropy', optimizer='sgd', metrics='accuracy')

In [11]:
# Get data

s1 = stateFactory.generate_complete_state().board
s2 = stateFactory.generate_complete_state().board
s3 = stateFactory.generate_complete_state().board

combined = [s1, s2, s3]
x_train = np.array(combined)
print(x_train.shape)

(3, 9, 9)


In [12]:
# Split training and test data

# TODO: Get it done!

In [14]:
# Fit model to training data
model.fit(x_train, y_train)

NameError: name 'y_train' is not defined